# Modelo de Riqueza
En este modelo se analiza una economía basada en agentes muy simple, extraída de la econofísica y que presenta un enfoque de mecánica estadística para la distribución de la riqueza [ [Dragulescu2002](https://arxiv.org/abs/cond-mat/0211175) ]. 

## Reglas
Las reglas de nuestro modelo son:
* Hay algunos agentes.
* Todos los agentes comienzan con 1 unidad de dinero.
* En cada paso del modelo, un agente entrega 1 unidad de dinero (si la tiene) a otro agente.

A pesar de su simplicidad, este modelo produce resultados que a menudo son inesperados para quienes no están familiarizados con él. Para nuestros propósitos, también demuestra fácilmente las funciones principales de Mesa.

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar a los integrantes de nuestro sistema usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes.

In [21]:
from mesa import Agent, Model
from mesa.time import RandomActivation

# For a jupyter notebook add the following line:
%matplotlib inline
# The below is needed for both notebooks and scripts
import matplotlib.pyplot as plt